In [52]:
import pandas as pd
import numpy as np

import configparser

In [73]:
dataset = pd.read_csv('../data/raw/train.csv')
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [85]:
config = configparser.ConfigParser()
config.read('../pipeline.cfg')
#valor = config.get('GENERAL', 'TARGET')

['../pipeline.cfg']

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.selection import DropFeatures

In [106]:
drop_vars = list(config.get('GENERAL', 'VARS_TO_DROP').split(', '))
#drop_vars.append(config.get('GENERAL', 'TARGET'))
drop_vars

['PassengerId', 'Name', 'Ticket', 'Cabin']

In [107]:
titanic_survived_model = Pipeline([
    # eliminamos columnas que no queremos.
    ('delete_features', DropFeatures(features_to_drop=drop_vars)),
    # imputación de variables continuas.
    ('continues_var_imputation', MeanMedianImputer(imputation_method='mean', variables=config.get('CONTINUES', 'VARS_TO_IMPUTE').split(', ') )),

    # imputacuón de variables categóricas
    ('categorical_var_imputation', CategoricalImputer(imputation_method='frequent', variables=config.get('CATEGORICAL', 'VARS_TO_IMPUTE'))),

    # codificación de variables categoricas
    ('categorical_encode_ohe', OneHotEncoder(drop_last=True, variables=config.get('CATEGORICAL', 'OHE_VARS'))),
    ('categorical_encode_frequency', CountFrequencyEncoder(encoding_method='count', variables=config.get('CATEGORICAL', 'FREQUENCY_ENC_VARS'))),

    # feature scaling
    ('fueature_scaling', StandardScaler())
])

In [ ]:
# corremos pipeline
#titanic_survived_model.fit(dataset)

Pipeline(steps=[('delete_features',
                 DropFeatures(features_to_drop=['PassengerId', 'Name', 'Ticket',
                                                'Cabin', 'Survived'])),
                ('continues_var_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['Age', 'Fare'])),
                ('categorical_var_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables='Embarked')),
                ('categorical_encode_ohe',
                 OneHotEncoder(drop_last=True, variables='Sex')),
                ('categorical_encode_frequency',
                 CountFrequencyEncoder(variables='Embarked')),
                ('fueature_scaling', StandardScaler())])

In [103]:
list(set(dataset.columns).difference(set(drop_vars)))

['Embarked', 'Sex', 'SibSp', 'Age', 'Parch', 'Pclass', 'Fare']

In [104]:
x_features_processed = titanic_survived_model.transform(dataset)
col_names = list(set(dataset.columns).difference(set(drop_vars)))
df_features_processed = pd.DataFrame(x_features_processed, columns=col_names)
df_features_processed['Survived'] = dataset['Survived']

df_features_processed.to_csv('../data/processed/features_for_models.csv', index=False)
df_features_processed.head()

NotFittedError: This DropFeatures instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [108]:
import pickle

with open('../artifacts/pipeline.pkl', 'wb') as f:
    pickle.dump(titanic_survived_model, f)